# Filter rooftop data to pilot districts
Import all of the s2 files and filter for rooftops within the pilot districts

**Purpose:**  


**Contents:**  
1. Import packages and set paths
2. Import SHRUG district boundary data and filter for pilot districts
3. 

## 1. Import packages and set paths

In [17]:
import geopandas as gpd
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from rooftop_tools.utils_rooftop import (
    get_overlapping_s2_cell_ids,
    get_matched_rooftop_centroids_from_s2_file
)


In [19]:
# set paths
FH_SAMPLING_FOLDER = Path('../')/'data'/'fortify_data'
SHRUG_district_path = FH_SAMPLING_FOLDER/'Shape files/shrug-pc11dist-poly-shp/district.shp'

## 2. Import SHRUG district boundary data and filter for pilot districts

In [20]:
# import SHRUG district boundary and select just the pilot districts
SHRUG_districts = gpd.read_file(SHRUG_district_path)
pilot_districts = SHRUG_districts[SHRUG_districts['d_name'].isin(['Katni', 'Sirmaur', 'Gandhinagar', 'Latur', 'Indore'])]
# print the gdf to make sure there were no spelling mistakes in district names
pilot_districts

,pc11_s_id,pc11_d_id,d_name,geometry
5,24,473,Gandhinagar,"POLYGON ((72.66212 23.56269, 72.66338 23.56259..."
95,23,439,Indore,"POLYGON ((75.60571 23.07781, 75.60621 23.0778,..."
106,23,450,Katni,"POLYGON ((80.71481 24.10525, 80.71287 24.10385..."
133,02,032,Sirmaur,"POLYGON ((77.40632 31.00785, 77.40671 31.00742..."
523,27,524,Latur,"MULTIPOLYGON (((77.32969 18.44447, 77.33087 18..."


## 3. For each s2 file, filter for rooftops in districts

In [21]:
s2_cell_ids = get_overlapping_s2_cell_ids(pilot_districts)
matched_rooftop_centroids_gdf_list = []

for s2_cell_id in tqdm(s2_cell_ids):
    matched_rooftop_centroids_gdf = get_matched_rooftop_centroids_from_s2_file(
        s2_file_dir=FH_SAMPLING_FOLDER / "Rooftop Data",
        s2_cell_id=s2_cell_id,
        boundaries_gdf=pilot_districts,
    )
    matched_rooftop_centroids_gdf_list.append(matched_rooftop_centroids_gdf)

/Users/douglasjohnson/Documents/code/rooftop_tools/src/rooftop_tools/utils_rooftop.py:124: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  points = gdf.geometry.centroid.to_frame(name="geometry")
/Users/douglasjohnson/Documents/code/rooftop_tools/src/rooftop_tools/utils_rooftop.py:137: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  points_new = leftover_shapes.geometry.centroid.to_frame(name="geometry")
/Users/douglasjohnson/Documents/code/rooftop_tools/src/rooftop_tools/utils_rooftop.py:137: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  points_new = leftover_shapes.geometr

Shapes with spillover after round 1: 4
Shapes with spillover after round 2: 1
Shapes with spillover after round 3: 1
Shapes with spillover after round 4: 0


  0%|          | 0/11 [00:00<?, ?it/s]/Users/douglasjohnson/Documents/code/rooftop_tools/src/rooftop_tools/utils_rooftop.py:177: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  s2_rooftops_gdf.geometry.centroid
  9%|▉         | 1/11 [00:16<02:40, 16.09s/it]/Users/douglasjohnson/Documents/code/rooftop_tools/src/rooftop_tools/utils_rooftop.py:177: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  s2_rooftops_gdf.geometry.centroid
 18%|█▊        | 2/11 [00:24<01:46, 11.80s/it]/Users/douglasjohnson/Documents/code/rooftop_tools/src/rooftop_tools/utils_rooftop.py:177: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projecte

In [25]:
# concatenate the gdfs
matched_rooftop_centroids_gdf = pd.concat(
    matched_rooftop_centroids_gdf_list, ignore_index=True
)
# Save the matched rooftops data
matched_rooftop_centroids_gdf.to_parquet(FH_SAMPLING_FOLDER/"Cleaned rooftop data"/ "Rooftops in 5 pilot dists.parquet")